In [25]:
# Importing necessary packages

import pandas as pd
import numpy as np
import re
import geocoder
from geopy.geocoders import Nominatim
import warnings
warnings.filterwarnings("ignore")

In [26]:
# Loading the parks excel file
# skipping first row and last 2107 rows which are not necessary
df = pd.read_excel('parks.xlsx',"DETAIL",skiprows=1,skipfooter = 2107)


# dropping the unrequired columns
df = df.drop(['Type', 'IUCN','Gazetted Area (ha)', 'Area (ha)', 'Declared','Latest Gazettal','IBRA Regions'], axis=1)
df.head(10)

,Name,TYPE ABBR,Longitude,Latitude
0,Bay of Islands Coastal Park,CP,142.711356,-38.514002
1,Cape Conran Coastal Park,CP,148.853934,-37.768376
2,Cape Liptrap Coastal Park,CP,145.870022,-38.770937
3,Discovery Bay Coastal Park,CP,141.302076,-38.214820
4,Gadsen Bend Park,CP,142.713632,-34.623129
5,Gippsland Lakes Coastal Park,CP,147.779805,-37.956123
6,Kings Billabong Park,CP,142.229986,-34.272019
7,Langwarrin Flora & Fauna Reserve,CP,145.176171,-38.173009
8,Murray - Kulkyne Park,CP,142.460501,-34.570468
9,Murray - Kulkyne Park,CP,142.581955,-34.780753


In [27]:
# checking for null values
print(df.isnull().sum())

Name          1
TYPE ABBR    17
Longitude    17
Latitude     17
dtype: int64


In [28]:
# Deleting the records where both latitude and longitude are null
df = df.dropna(axis=0, subset=['Latitude','Longitude'])

# deleting the records where name column is null
df = df.dropna(axis=0, subset=['Name'])

In [29]:
# checking for null values
print(df.isnull().sum())

Name         0
TYPE ABBR    0
Longitude    0
Latitude     0
dtype: int64


In [30]:
# creating a list called park_type to store the necessary park types
park_type = ['CP','NP','NCR','NFR','OP','PNPA','SP']

# only storing the necessary park type records in the dataframe
df= df.loc[df['TYPE ABBR'].isin(park_type)]
df

,Name,TYPE ABBR,Longitude,Latitude
0,Bay of Islands Coastal Park,CP,142.711356,-38.514002
1,Cape Conran Coastal Park,CP,148.853934,-37.768376
2,Cape Liptrap Coastal Park,CP,145.870022,-38.770937
3,Discovery Bay Coastal Park,CP,141.302076,-38.214820
4,Gadsen Bend Park,CP,142.713632,-34.623129
5,Gippsland Lakes Coastal Park,CP,147.779805,-37.956123
6,Kings Billabong Park,CP,142.229986,-34.272019
7,Langwarrin Flora & Fauna Reserve,CP,145.176171,-38.173009
8,Murray - Kulkyne Park,CP,142.460501,-34.570468
9,Murray - Kulkyne Park,CP,142.581955,-34.780753


In [31]:
# replacing substrings like F.R to Flora Reserve, W.R to Wildlife Reserve
df['Name'].replace({'F.F.R': 'Flora and Fauna Reserve','N.F.R':'Natural Features Reserve','N.F.S.R':'Natural Features and Scenic Reserve'}, inplace=True, regex=True)
df['Name'].replace({'SS.R': 'Streamside Reserve','S.R':'Scenic Reserve','G.R':'Geological Reserve','B.R':'Bushland Reserve'}, inplace=True, regex=True)
df['Name'].replace({'F.R': 'Flora Reserve','W.R':'Wildlife Reserve','N.C.R':'Nature Conservation Reserve'}, inplace=True, regex=True)

In [32]:
# Adding substring called "State Park" to the parks which are of "SP" Type
df['Name'] = np.where(df['TYPE ABBR'].eq('SP'), df['Name'] + 'State Park', df['Name'])

In [33]:
# dropping the TYPE ABBR column
df = df.drop(['TYPE ABBR'], axis=1)

In [34]:
#Exporting to csv file
df.to_csv('park_file.csv',index = False)

In [35]:
#Loading the park_file csv file
df = pd.read_csv('park_file.csv')

# creating a dataframe df1 with only latitude and longitude columns
df1 = df[['Latitude','Longitude']]

# showing only first 5 rows
df1.head(5)

,Latitude,Longitude
0,-38.514002,142.711356
1,-37.768376,148.853934
2,-38.770937,145.870022
3,-38.214820,141.302076
4,-34.623129,142.713632


In [2]:
# reverse geocoding using geolocator.reverse to get the address from latitude and longitude
geolocator = Nominatim(user_agent="monash university research project_moms care")

df1['address'] = df1.apply(lambda row: geolocator.reverse((row['Latitude'], row['Longitude']),timeout=15), axis=1)
df1

NameError: name 'Nominatim' is not defined

In [110]:
#Exporting to csv file
df1.to_csv('park_address_extract.csv',index = False)

In [36]:
#Loading the toilet_file_extract csv file
df1 = pd.read_csv('park_address_extract.csv')
df1.head(10)

,Latitude,Longitude,address
0,-38.514002,142.711356,"Bay of Islands CP, Stanhopes Bay Road, Mepunga..."
1,-37.768376,148.853934,"Bemm River, Shire of East Gippsland, Gippsland..."
2,-38.770937,145.870022,"Tarwin Lower, Shire of South Gippsland, Gippsl..."
3,-38.214820,141.302076,"Drumborg, Shire of Glenelg, Barwon South West,..."
4,-34.623129,142.713632,"Gadsen Bend Park, Malaya Road, Rural City of S..."
5,-37.956123,147.779805,"Ocean Grange, Loch Sport, Shire of Wellington,..."
6,-34.272019,142.229986,"Kings Billabong Park, Woorlong Avenue, Red Cli..."
7,-38.173009,145.176171,"Langwarrin FFR, Centre Break, Langwarrin, Fran..."
8,-34.570468,142.460501,"Hattah, Rural City of Mildura, Loddon Mallee, ..."
9,-34.780753,142.581955,"Wemen, Rural City of Swan Hill, Loddon Mallee,..."


In [37]:
# splitting the address column into address,country and postcode columns
df1[['address','country']] = df1['address'].str.rsplit(pat=',',expand=True, n=1)

In [38]:
df1[['address','postcode']] = df1['address'].str.rsplit(pat=',',expand=True, n=1)
df1[['address','state']] = df1['address'].str.rsplit(pat=',',expand=True, n=1)
df1[['address','city council']] = df1['address'].str.rsplit(pat=',',expand=True, n=1)
df1[['address','suburb']] = df1['address'].str.rsplit(pat=',',expand=True, n=1)
df1.head(20)

,Latitude,Longitude,address,country,postcode,state,city council,suburb
0,-38.514002,142.711356,"Bay of Islands CP, Stanhopes Bay Road, Mepunga...",Australia,3268,Victoria,Barwon South West,Shire of Moyne
1,-37.768376,148.853934,Bemm River,Australia,3889,Victoria,Gippsland,Shire of East Gippsland
2,-38.770937,145.870022,Tarwin Lower,Australia,Victoria,Gippsland,Shire of South Gippsland,None
3,-38.214820,141.302076,Drumborg,Australia,Victoria,Barwon South West,Shire of Glenelg,None
4,-34.623129,142.713632,Gadsen Bend Park,Australia,Victoria,Loddon Mallee,Rural City of Swan Hill,Malaya Road
5,-37.956123,147.779805,Ocean Grange,Australia,Victoria,Gippsland,Shire of Wellington,Loch Sport
6,-34.272019,142.229986,"Kings Billabong Park, Woorlong Avenue, Red Cliffs",Australia,Victoria,Loddon Mallee,Rural City of Mildura,Mildura
7,-38.173009,145.176171,"Langwarrin FFR, Centre Break, Langwarrin",Australia,3910,Victoria,City of Frankston,Frankston
8,-34.570468,142.460501,Hattah,Australia,Victoria,Loddon Mallee,Rural City of Mildura,None
9,-34.780753,142.581955,Wemen,Australia,Victoria,Loddon Mallee,Rural City of Swan Hill,None


In [39]:
# joining suburb into address column
df1['address'] = df1['address'] + ',' + df1['suburb']
df1

,Latitude,Longitude,address,country,postcode,state,city council,suburb
0,-38.514002,142.711356,"Bay of Islands CP, Stanhopes Bay Road, Mepunga...",Australia,3268,Victoria,Barwon South West,Shire of Moyne
1,-37.768376,148.853934,"Bemm River, Shire of East Gippsland",Australia,3889,Victoria,Gippsland,Shire of East Gippsland
2,-38.770937,145.870022,NaN,Australia,Victoria,Gippsland,Shire of South Gippsland,None
3,-38.214820,141.302076,NaN,Australia,Victoria,Barwon South West,Shire of Glenelg,None
4,-34.623129,142.713632,"Gadsen Bend Park, Malaya Road",Australia,Victoria,Loddon Mallee,Rural City of Swan Hill,Malaya Road
5,-37.956123,147.779805,"Ocean Grange, Loch Sport",Australia,Victoria,Gippsland,Shire of Wellington,Loch Sport
6,-34.272019,142.229986,"Kings Billabong Park, Woorlong Avenue, Red Cli...",Australia,Victoria,Loddon Mallee,Rural City of Mildura,Mildura
7,-38.173009,145.176171,"Langwarrin FFR, Centre Break, Langwarrin, Fran...",Australia,3910,Victoria,City of Frankston,Frankston
8,-34.570468,142.460501,NaN,Australia,Victoria,Loddon Mallee,Rural City of Mildura,None
9,-34.780753,142.581955,NaN,Australia,Victoria,Loddon Mallee,Rural City of Swan Hill,None


In [40]:
# filling the null values of postcode column
# groupby suburb, and transform('first') 
df1['postcode'] = df1.groupby("suburb")["postcode"].transform("first")

In [41]:
# dropping the unrequired columns
#df2 = df2.drop(['country', 'state','city council','suburb', ], axis=1)
#df2.head(10)

In [42]:
# creating a dataframe df2 with only place name column
df2 = df[['Name']]

In [43]:
# merging two dataframes df1 and df2
df3=df1.join(df2)
df3

,Latitude,Longitude,address,country,postcode,state,city council,suburb,Name
0,-38.514002,142.711356,"Bay of Islands CP, Stanhopes Bay Road, Mepunga...",Australia,3268,Victoria,Barwon South West,Shire of Moyne,Bay of Islands Coastal Park
1,-37.768376,148.853934,"Bemm River, Shire of East Gippsland",Australia,3889,Victoria,Gippsland,Shire of East Gippsland,Cape Conran Coastal Park
2,-38.770937,145.870022,NaN,Australia,NaN,Gippsland,Shire of South Gippsland,None,Cape Liptrap Coastal Park
3,-38.214820,141.302076,NaN,Australia,NaN,Barwon South West,Shire of Glenelg,None,Discovery Bay Coastal Park
4,-34.623129,142.713632,"Gadsen Bend Park, Malaya Road",Australia,Victoria,Loddon Mallee,Rural City of Swan Hill,Malaya Road,Gadsen Bend Park
5,-37.956123,147.779805,"Ocean Grange, Loch Sport",Australia,Victoria,Gippsland,Shire of Wellington,Loch Sport,Gippsland Lakes Coastal Park
6,-34.272019,142.229986,"Kings Billabong Park, Woorlong Avenue, Red Cli...",Australia,Victoria,Loddon Mallee,Rural City of Mildura,Mildura,Kings Billabong Park
7,-38.173009,145.176171,"Langwarrin FFR, Centre Break, Langwarrin, Fran...",Australia,3910,Victoria,City of Frankston,Frankston,Langwarrin Flora & Fauna Reserve
8,-34.570468,142.460501,NaN,Australia,NaN,Loddon Mallee,Rural City of Mildura,None,Murray - Kulkyne Park
9,-34.780753,142.581955,NaN,Australia,NaN,Loddon Mallee,Rural City of Swan Hill,None,Murray - Kulkyne Park


In [44]:
# deleting the records where name column is null
df3 = df3.dropna(axis=0, subset=['address'])

In [45]:
# checking for null values
print(df3.isnull().sum())

Latitude        0
Longitude       0
address         0
country         0
postcode        0
state           0
city council    0
suburb          0
Name            0
dtype: int64


In [46]:
# converting postcode to int datatype and filling null values with '0'
df3.postcode = pd.to_numeric(df3.postcode, errors='coerce').fillna(0).astype(np.int64)
df3['postcode'].dtype
df3['Latitude'].dtype

dtype('float64')

In [47]:
# dropping the suburb column
df3 = df3.drop(['country','state','city council','suburb'], axis=1)

In [48]:
#Renaming columns
df3.columns = ['lat', 'long', 'address', 'postcode','name']

In [49]:
df3 = df3.drop_duplicates(subset=['name'], keep='first')
df3

,lat,long,address,postcode,name
0,-38.514002,142.711356,"Bay of Islands CP, Stanhopes Bay Road, Mepunga...",3268,Bay of Islands Coastal Park
1,-37.768376,148.853934,"Bemm River, Shire of East Gippsland",3889,Cape Conran Coastal Park
4,-34.623129,142.713632,"Gadsen Bend Park, Malaya Road",0,Gadsen Bend Park
5,-37.956123,147.779805,"Ocean Grange, Loch Sport",0,Gippsland Lakes Coastal Park
6,-34.272019,142.229986,"Kings Billabong Park, Woorlong Avenue, Red Cli...",0,Kings Billabong Park
7,-38.173009,145.176171,"Langwarrin FFR, Centre Break, Langwarrin, Fran...",3910,Langwarrin Flora & Fauna Reserve
10,-35.133470,143.370406,"Nyah-Vinifera Park, River Track, Nyah, Wood Wo...",3595,Nyah-Vinifera Park
11,-37.619382,148.223227,"Mottle Range FR, Monument Track, Buchan",0,Tara Range Park
13,-38.124196,146.431873,"Wirilda Walking Track, Tyers, Traralgon",0,Tyers Park
15,-37.031704,147.336706,"Dungey Track (overgrown), Harrietville",0,Alpine National Park


In [50]:
#Exporting to csv file
df3.to_csv('park_vic.csv',index = False)

In [51]:
#Loading the park_file csv file
df3 = pd.read_csv('parks_victoria.csv')

In [52]:
df3 = df3[df3.postcode != 0]

In [53]:
#Exporting to csv file
df3.to_csv('parks_victoria_final.csv',index = False)